In [ ]:
BATCH_SIZE = 32
IMG_SHAPE=(460, 460, 1)
BASE_DIR = "../processed/"

In [ ]:
import os
import pandas as pd
from covidxpert import load_images
from covidxpert import load_keras_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load the data

In [ ]:
df = pd.read_csv("../datasets/normalized_index/df_all_common_cols.csv")
# temporary workaround
tf = df[ df.img_path.str.endswith(".jpeg") | df.img_path.str.endswith(".png") ]
tf

In [ ]:
filenames = [
    os.path.join(BASE_DIR, x) 
    for x in tf.img_path.values
]
labels = list(tf.covid19.values)

In [ ]:
training_data = load_images(filenames, labels, image_size=IMG_SHAPE, batch_size=BATCH_SIZE)

# Train the model

In [ ]:
from tensorflow.keras.applications import ResNet50
model = load_keras_model(ResNet50, IMG_SHAPE)
model.summary()

In [ ]:
history = model.fit(
    training_data,
    epochs=1000,
    callbacks=[
        EarlyStopping(
            monitor="loss",
            patience=10,
            min_delta=0.001,
            restore_best_weights=False
        ),
        ReduceLROnPlateau(
            monitor="loss",
            patience=5,
            min_delta=0.001
        )
    ]
).history